In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [28]:
df_words = pd.read_csv('words_to_translate.csv', sep=';')
df_words_test = df_words[:5]
df_words_test

,words,nature,translation
0,to (kid) themselves,v,NaN
1,the weeks crept on (crept),v,NaN
2,turned it (thrice) in his hand,NaN,NaN
3,maddeningly,NaN,NaN
4,the (strain) of not laughing,n,NaN


In [52]:

personal_pronouns = ['i', 'you', 'he', 'she', 'it', 'we', 'they']
indefinite_pronouns = ['sth', 'sb']
to = 'to'

def operation(row):
    nature = ''
    if pd.isnull(row['nature']) is False:
        nature = row['nature'].lower()
    words = row['words'].lower()
    word = ''
    if ('(' in words) & (')' in words):
        pattern = r"\((.*?)\)"
        word = re.findall(pattern, words)[0]
    else:
        words = words.split(' ')
        if len(words) == 1:
            word = words[0]
        elif len(words) == 2:
            for i in range(2):
                for personal_pronoun in personal_pronouns:
                    if personal_pronoun == words[i]:
                        word = words[abs(i-1)]
                for indefinite_pronoun in indefinite_pronouns:
                    if indefinite_pronoun == words[i]:
                        indef_pronoun = '['+words[i]+']'
                        word = words[abs(i-1)]
                if to == words[i]:
                    word = words[abs(i-1)]
                if nature != '':
                    nature = 'v'
        elif len(words) > 2:
            print('------------------------------------------------------')
            print('------------------cant be dealt with-------------------')
            print(words)
            print('------------------------------------------------------')

    translations = get_translation(word)
    filtered_translations = []
    print('SELECTION')
    print(translations)
    if nature != '':
        if nature == 'v':
            for i in range(len(translations)):
                if nature == translations[i][1][0]:
                    filtered_translations.append(translations[i])
        else:
            for i in range(len(translations)):
                if nature == translations[i][1]:
                    filtered_translations.append(translations[i])
    print('SELECTED')
    for translation in filtered_translations:
        print(translation)
        print()
    print()
    return 'translated'

In [53]:
df_words_test['translation'] = df_words_test.apply(operation, axis=1)

SELECTION
[['kid', 'n', 'informal(child)', '(familier)gamin, gamine ', 'enfant ', '(familier, France, Belg, Suisse)gosse ', 'Stacy likes to visit with friends while the kids are at school.', "Stacy aime rendre visite à des amis lorsque les gamins sont à l'école."], ['kid', 'n', '(young goat)', 'chevreau ', 'The goat gave birth to a kid last month.', 'La chèvre a donné naissance à un chevreau le mois dernier.'], ['kid', 'adj', 'frère, sœur(younger)', '(frère, sœur)petit, petite ', 'Ben took his kid sister to school.', "Ben a emmené sa petite sœur à l'école."], ['kid', 'vi', 'informal(joke)', 'plaisanter, rigoler⇒ ', '(familier)blaguer ', "I didn't mean it; I was just kidding!"], ['kid [sb]', 'vtr', '(joke)', 'taquiner ', '(familier)faire marcher ', '(familier)charrier ', 'se moquer de ', 'Karen made a face and said, "You\'ve got to be kidding me!"', 'Karen fit une grimace et dit "Tu me fais marcher, là !"', 'Karen fit une grimace et dit "Tu me charries, là !"', 'Karen fit une grimace et

C:\Users\charl\AppData\Local\Temp\ipykernel_18932\2704481587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_words_test['translation'] = df_words_test.apply(operation, axis=1)


In [26]:
def get_translation(word):
    r = requests.get('https://www.wordreference.com/enfr/'+word)
    soup = BeautifulSoup(r.text, 'html.parser')
    test_array = (soup.find_all('table', class_='WRD')[0].find_all('tr', class_=['even', 'odd']))

    translation_blocks = []
    block_temp = []
    for i in range(len(test_array)):
        if i != 0:
            if test_array[i].attrs.get('class')[0] != test_array[i - 1].attrs.get('class')[0]:
                translation_blocks.append(block_temp)
                block_temp = []
        block_temp.append(test_array[i].find_all('td'))
    translation_blocks.append(block_temp)

    array_translations = []
    array_temp = []

    for block in translation_blocks:
        if block[0][0].find('em'): # check for nature
            nature = block[0][0].find('em')
            nature.extract()
        if block[0][0].find('a'): # remove a tag from english
            block[0][0].find('a').extract()
        word = block[0][0].get_text()
        figurative = ''
        if block[0][1].find('i'):
            figurative = block[0][1].find('i').get_text()
        definition = figurative+''.join(block[0][1].find_all(string=True, recursive=False)).strip()
        array_temp.append(word.strip())
        array_temp.append(nature.get_text())

        traduction_context = block[0][1].find('span', class_='dsense')

        if definition != '':
            array_temp.append(definition.replace('\xa0', '').strip())
        for y in range(len(block)):
            if block[y][1].find('span', class_='dsense'):
                traduction_context = block[y][1].find('span', class_='dsense').get_text()
                if block[y][2].find('em'):
                    block[y][2].find('em').extract()
                if block[y][2].find('a'):
                    block[y][2].find('a').extract()
                traduction = block[y][2].get_text()
                array_temp.append(traduction_context+traduction)
            elif block[y][1].has_attr('class') and 'FrEx' in block[y][1]['class']: # example
                array_temp.append(block[y][1].get_text())
            elif block[y][1].has_attr('class') and 'ToEx' in block[y][1]['class']: # example
                array_temp.append(block[y][1].get_text())
            else:  #
                if block[y][2].find('em'):
                    block[y][2].find('em').extract()
                if block[y][2].find('a'):
                    block[y][2].find('a').extract()
                traduction = block[y][2].get_text()
                array_temp.append(traduction)
        array_translations.append(array_temp)
        array_temp = []

    return array_translations